In [209]:
from numpy import *

# Higgs Physics Marked Exercise

## Question 1: Calculate the branching ratios for Higgs boson decays to photons, gluons, 4th gen neutrinos, tau and bottom quarks.

### Background
To determine the branching ratios of these Higgs decays, we first need to calculate the partial decay widths of all the potential Higgs decays. By summing these partial decay widths we find the total decay width of the Higgs boson. The branching ratio of a particular decay is then given by the partial decay width of that decay divided by the total decay width of the Higgs boson. To calculate the partial widths of the decays we square the matrix element, sum over polarisations and colours (if relevant), then integrate over the final state two body phase space. Doing this for Higgs decays to fermions, photons and gluons gives the partial decay widths as:
$$
\Gamma(h \rightarrow f \bar{f})=\frac{N_{c}}{8 \pi} \frac{m_{f}^{2}}{v^{2}} m_{h}\left[1-\frac{4 m_{f}^{2}}{m_{h}^{2}}\right]^{3 / 2}
$$
$$
\Gamma(h \rightarrow \gamma \gamma)=\frac{\alpha^{2}}{256 \pi^{3}} \frac{m_{h}^{3}}{v^{2}}\left|\sum_{i} N_{c i} Q_{i}^{2} F_{i}\left(\tau_{i}\right)\right|^{2}
$$
$$
\Gamma(h \rightarrow g g)=\frac{\alpha_{s}^{2}}{256 \pi^{3}} \frac{m_{h}^{3}}{v^{2}} \times 2\left|\sum_{i} F_{1 / 2}\left(\tau_{i}\right)\right|^{2}
$$
respectively. $N_{c}$ accounts for the colour factor and is 3 for quarks and 1 for leptons, $m_{f}$ is the mass of the fermion, $v$ is the vacuum expectation energy, $m_{h}$ is the Higgs mass, $\alpha$ is the fine structure constant for electromagnetism (~ 1/137), $\alpha_{s}$ is the fine structure constant for the strong interaction (~ 1/11), $Q$ is electric charge, $\tau = \frac{4m_{i}^{2}}{m_{h}^{2}}$ and F is a function which calculates the loop factors given $\tau$. The sum runs over all particles which could take part in the loop. For the decay to photons this is all charged particles, and for decays to gluons this is only the quarks (due to colour).
We achieve the matrix element used to derive these formulas from terms in the Yukawa interaction Lagrangian. This gives us a coefficient when the particles couple to the Higgs and this coefficient is the vertex amplitude. Using Feynman rules we can construct matrix elements such as
$ iM_{f} = \bar{u_{f}} \frac{-im_{f}}{v} v_{\bar{f}} $ for fermions.

In [210]:
# Define constants used throughout
a = 1/137  # Fine structure constant
a_s = 0.09445 # Strong fine structure constant for Higgs mass: http://hyperphysics.phy-astr.gsu.edu/hbase/Forces/couple.html
m_h = 125000  # MeV
v = 246219.6  # MeV
# This value is constant for a given Higgs mass and appears in both equations for calculating decays to photons
# and gluons, thus I have defined it here for conciseness
const = ((m_h**3)) / (256 * (pi**3) * (v**2))

# This is a dictionary of all the particles in the standard model plus the 4th gen of fermions
particles = {
    ####### mass is given in MeV and charge is given in units of e
    #######         gen, type,   mass, e charge
    "up"         : ["1", "quark", 2.2, 2/3],
    "down"       : ["1", "quark", 4.7, -1/3],
    "e"          : ["1", "lepton", 0.511, -1],
    "v_electron" : ["1", "lepton", 2.2*(10**-6), 0],
    "charm"      : ["2", "quark", 1280, 2/3],
    "strange"    : ["2", "quark", 95, -1/3],
    "muon"       : ["2", "lepton", 105.7, -1],
    "v_muon"     : ["2", "lepton", 0.17, 0],
    "top"        : ["3", "quark", 173000, 2/3],
    "bottom"     : ["3", "quark", 4180, -1/3],
    "tau"        : ["3", "lepton", 1777, -1],
    "v_tau"      : ["3", "lepton", 15.5, 0],
    "up_4"       : ["4", "quark", 500000, 2/3],  # 4th generation starts
    "down_4"     : ["4", "quark", 400000, -1/3],
    "l_4"        : ["4", "lepton", 100000, -1],
    "v_4"        : ["4", "lepton", 60000, 0],  # 4th generation ends
    "gluon"      : [None, "boson", 0, 0],
    "photon"     : [None, "boson", 0, 0],
    "W+"         : [None, "boson", 80400, 1],
    "W-"         : [None, "boson", 80400, -1],
    "Z"          : [None, "boson", 91200, 0],
    "H"          : [None, "boson", 125000, 0]
}

In [211]:
''' This function calculates the partial decay width for Higgs decays to fermions '''
def fermion_width():
    
    sm_names = list(particles.keys()) # Get the names of the particles as a list
    sm = list(particles.values()) # Get the values of the particles as a list of lists
    for i, _ in enumerate(particles.values()):
        particle_type = sm[i][1]  # The ith item in the list is the lost of current particle properties, the next index is the desired property.
        if particle_type != "boson":  # Calculate the decay width for all fermions
            mass = sm[i][2]  # Get particle mass
            particle = sm_names[i]  # Get particle name
            if particle_type == "quark": N = 3  # Define colour constant
            else: N = 1
            
            width = (mass**2 * N * m_h * (1 - (2*mass/m_h)**2)**(3/2)) / (8 * pi * v**2)  # Calculate the width
            particle_widths.update({particle : width})  # Append results to our decay widths dictionary

In [212]:
''' The functions below calculate the loop factors in the Higgs decays to gluons and photons '''
def F(tau, m, spin):
    f_tau = f(tau, m)
    if spin == 1:     return 2 + 3*tau + 3*tau*(2-tau)*f_tau  # If the particle is a boson
    elif spin == 1/2: return -2*tau * (1 + (1-tau)*f_tau)  # If the particle is a fermion
    else:             print("Error")

def f(tau, m):
    if m_h <= 2*m:  return (arcsin(1/sqrt(tau)))**2  # If the mass of the decay products is greater than the Higgs mass
    elif m_h > 2*m: return ((log(1 + sqrt(1-tau)) - log(1 - sqrt(1-tau)) - 1j*pi)**2) * (-1/4)  # If Higgs mass is greater than mass of the
    else:           print("Error")                                                              # decay products

In [213]:
''' This function calculates the partial decay width of Higgs decays to photons at 1-loop '''
def photon_width():
    
    val = 0  # Variable to be added to for each sum
    sm = list(particles.values())  # Get particle properties as list of lists
    for i, _ in enumerate(particles.values()): # Iterate over each particle in the dictionary
        
        Q = sm[i][3]
        if Q != 0:  # Only charged particles contribute so only perform calculations for those where Q != 0
            
            # Assigning variables
            mass = sm[i][2]  # Assign mass
            tau = (2*(mass/m_h))**2  # Calculate tau
            if sm[i][1] == "boson": spin = 1  # Assign spin
            else: spin = 1/2
            if sm[i][1] == "quark": N = 3  # Assign colour constant
            else: N = 1

            # Calculating width
            val += N * (Q**2) * F(tau, mass, spin)
        
    width = (a**2) * const * ((abs(val))**2) # Calculate width
    particle_widths.update({"photon" : width}) # Append to our decay widths dictionary

In [214]:
''' This function calculates the partial decay width of Higgs decays to gluons at 1-loop '''
def gluon_width():
    
    val = 0  # Variable to be added to for each sum
    sm = list(particles.values())  # Get particle properties as list of lists
    for i, _ in enumerate(particles.values()):  # Iterate over each particle in the dictionary
        
        particle_type = sm[i][1]  # Only quarks in the sum
        if particle_type == "quark":  # So only perform calculation for quarks

            # Assigning values
            mass = sm[i][2]  # Assign mass variable
            tau = (2 * (mass / m_h))**2  # Calculate tau
            
            # Calculating width
            val += F(tau, mass, 1/2)  # Increase val each iteration

    width = (a_s**2) * const * 2 * ((abs(val))**2)  # calculate the width
    particle_widths.update({"gluon" : width}) # Update our dictionary of widths

In [215]:
''' This function creates a dictionary and calls each of the above functions in order to populate it with each particle and the Higgs
decay width to that particle. It also uses the assumptions given for the decay width to W and Z bosons'''
def main():
    
    global particle_widths  # Make global so available within the functions which append to it
    particle_widths = {} # Initialise as empty
    
    fermion_width() # Populate with decay width of higgs to all fermions
    photon_width() # Populate with decay width of higgs to photons
    gluon_width() # Populate with decay width of higgs to gluons
    particle_widths.update({"W" : 0.8}) # Populate with given assumption for W
    particle_widths.update({"Z" : 0.01}) # Populate with given assumption for Z
    
    return particle_widths

widths = main() # Call the function to generate the dictionary of all particles and the decay width of higgs to them

In [216]:
# Remove decays to particles with 2m_f > m_h as these are not energetically possible and so have given complex partial decay widths
particles_to_remove = ["top", "up_4", "down_4", "l_4"]
for i in particles_to_remove:
    try: del widths[i]
    except: pass

In [217]:
# Calculate the total decay width by summing all the partial decay widths
totalDecayWidth = sum(list(widths.values()))

In [218]:
''' This function calculates the branching ratio of all the particles we have calculated decay widths for '''
def branchingCalc(decayWidths):

    branches = {}  # Initialise dictionary to store results in
    
    particle_names = list(decayWidths.keys()) # Get names to use as keys
    widths = list(decayWidths.values()) # Get widths
    for i, _ in enumerate(decayWidths.values()): # Iterate over all the decay widths
        branching = widths[i] / totalDecayWidth # Calculate the branching ratio
        name = particle_names[i] # Get particle name
        branches.update({name : branching})  # Append particle name and branching ratio to dictionary as key:value pair

    return branches

branching_ratios = branchingCalc(widths)  # Call function to generate dictionary of decay widths.

In [219]:
''' In order to answer the actual question, I print the branching ratio of the requested decay channels as a percentage '''
requested_ratios = ["photon", "gluon", "v_4", "tau", "bottom"]  # List of particles I need to print the branching ratio of
for particle in requested_ratios:  # Iterate over the above list, printing branching ratio for each one.
    print("Branching ratio of Higgs boson decay to {0} is {1:.2f}%".format(particle, branching_ratios[particle]*100))

Branching ratio of Higgs boson decay to photon is 0.20%
Branching ratio of Higgs boson decay to gluon is 8.52%
Branching ratio of Higgs boson decay to v_4 is 48.39%
Branching ratio of Higgs boson decay to tau is 1.93%
Branching ratio of Higgs boson decay to bottom is 31.88%


The branching ratio of Higgs boson decays to photon and gluon pairs remains roughly unchanged in the case of a fourth chiral generation of fermions. In the SM, the branching ratio to photons is 0.23%, here I have calculated 0.20%; the branching ratio to gluons is 8.5%, here I have calculated exactly the same value to 1 decimal place. This is because the introduction of a fourth chiral generation of fermions does not interfere with Higgs decays to either of these particles. However, Higgs boson decays to fermions have been affected by the introduction of a chiral fourth generation. Decays to the 4th generation neutrino dominate at 48.4% of Higgs decays, pushing the previously dominating decay to bottom quarks from 56% down to 31.9%. Decays to tau have also dropped from 6.2% to 1.9%.

## Question 2: Name the three channels in which the Higgs boson has been discovered. In presence of a fourth generation, which mode would be the most sensitive to discover the Higgs boson?

The Higgs boson has been discovered via the channels $h \rightarrow \gamma\gamma$, $h \rightarrow ZZ^{*} \rightarrow \ell \bar{\ell}$ $\ell \bar{\ell}$ and $ h \rightarrow WW^{*} \rightarrow \ell v$ $\ell v $. In the presence of a fourth generation, $h \rightarrow v_{4} \bar{v_{4}}$ would be the most sensitive as it accounts for ~48.4% of decays, as calculated in question 1.

## Question 3: In an alternative electroweak symmetry breaking scenario, the Higgs potential is given by:
$$
V(\Phi) = \mu^{2} \Phi^{\dagger} \Phi+\frac{c_{8}}{\Lambda^{4}}\left(\Phi^{\dagger} \Phi\right)^{4}
$$
## Calculate a) the vacuum expectation value $v$ in terms of $\mu^{2}$, $c_{8}$ and $\Lambda$ and b) the Higgs boson mass, $m_{h}$, in terms of $\mu^{2}$, $c_{8}$ and $\Lambda$.

<center>To calculate the vacuum expectation value, $v$, we need to find the minimum of the potential by differentiating
$$
\frac{dV(\Phi)}{d\Phi^{\dagger}\Phi} = \mu^{2} + \frac{4c_{8}}{\Lambda^{4}}(\Phi^{\dagger}\Phi)^{3}
$$
<center>At the minimum, where $\frac{dV(\Phi)}{d\Phi^{\dagger}\Phi} = 0$, $(\Phi^{\dagger}\Phi)_{min} = (\frac{-\mu^{2}\Lambda^{4}}{4c_{8}})^{\frac{1}{3}}$
<center>
Given
$
\Phi=\left(\begin{array}{c} \phi^{+} \\ \phi^{0} \end{array}\right)=\frac{1}{\sqrt{2}}\left(\begin{array}{c} \phi_{1}+i \phi_{2} \\ \phi_{3}+i \phi_{4} \end{array}\right)
$
we find that
$
\Phi^{\dagger} \Phi=\frac{1}{2}\left(\phi_{1}^{2}+\phi_{2}^{2}+\phi_{3}^{2}+\phi_{4}^{2}\right)
$
we want $ \Phi^{\dagger} \Phi = (\Phi^{\dagger}\Phi)_{min}$, therefore:
</center>
$$
\frac{1}{2}\left(\phi_{1}^{2}+\phi_{2}^{2}+\phi_{3}^{2}+\phi_{4}^{2}\right) = (\frac{-\mu^{2}\Lambda^{4}}{4c_{8}})^{\frac{1}{3}}
$$
We can choose our basis of states to be oriented however we like. So we choose
$
\left\langle\phi_{1}\right\rangle=\left\langle\phi_{2}\right\rangle=\left\langle\phi_{4}\right\rangle=0$ and $
\left\langle\phi_{3}\right\rangle \equiv v
$
$$
\therefore \frac{1}{2}v^{2} = (\frac{-\mu^{2}\Lambda^{4}}{4c_{8}})^{\frac{1}{3}} \Longrightarrow v = (\frac{-2\mu^{2}\Lambda^{4}}{c_{8}})^{\frac{1}{6}}
$$

<center>
To calculate the Higgs boson mass we define a new scalar field $h$ with vacuum expectation energy $0$ according to $\phi_{3} = h + v$. Our field then becomes $\Phi=\frac{1}{\sqrt{2}}\left(\begin{array}{c}\phi_{1}+i \phi_{2} \\ v+h+i \phi_{4}\end{array}\right)$. Plugging this into our expression for the potential:
$$
V=\frac{\mu^{2}}{2}\left(\phi_{1}^{2}+\phi_{2}^{2}+(h+v)^{2}+\phi_{4}^{2}\right)+\frac{c_{8}}{4\Lambda^{4}}\left(\phi_{1}^{2}+\phi_{2}^{2}+(h+v)^{2}+\phi_{4}^{2}\right)^{4}
$$
We are only interested in terms quadratic in $h$ as those are our mass terms, so we set all other fields equal to their vev of 0 and expand out the brackets, keeping only terms proportional to $h^{2}$:
$$
V \supset \frac{1}{2} \mu^{2} h^{2} + \frac{c_{8}}{4\Lambda^{4}}(3v^{6}h^{2}+25v^{6}h^{2}) = \frac{1}{2} \mu^{2} h^{2} + \frac{7c_{8}}{\Lambda^{4}}v^{6}h^{2} = \frac{1}{2} \mu^{2} h^{2} + \frac{7c_{8}}{\Lambda^{4}} (\frac{-2\mu^{2}\Lambda^{4}}{c_{8}}) h^{2} = \frac{1}{2} \mu^{2} h^{2} - 14\mu^{2}h^{2} = -\frac{27}{2}\mu^{2}h^{2}
$$
Bosonic mass terms such as the Higgs have the form $ \frac{1}{2}m_{h}h^{2} $
$$ \therefore m_{h} = 3i\sqrt{3}\mu $$
Which is real and positive when $\mu$ is complex, i.e. when $\mu^{2} < 0$. If we had written the potential with a negative $\mu$ initially, the answer would be identical without the factor of $i$.